Digital Twin of A Thermostat creating a closed-loop control system. T

    A PhysicalRoom class to better simulate the physical world, including its own thermal dynamics (heating, cooling, and natural drift towards an ambient temperature).

    A set_point in the ThermostatTwin, representing the desired target temperature.

    Control logic within the twin (get_control_command) that uses hysteresis (a dead-band) to decide when to turn the heater or A/C on or off, preventing rapid cycling.

    A feedback loop where the twin's command is sent back to the PhysicalRoom to influence its state, thus creating a true control system.

In [1]:
import random
import time

In [2]:

# --- THERMAL DYNAMIC SIMULATION (THE PHYSICAL WORLD) ---

class PhysicalRoom:
    """
    Simulates the physical room temperature and the effect of HVAC.
    This is the 'Physical Asset' your twin is monitoring and controlling.
    """
    def __init__(self, initial_temp, ambient_temp):
        self.current_temp = initial_temp
        self.ambient_temp = ambient_temp
        self.heater_on = False
        self.ac_on = False
        
        # Thermal parameters
        self.HEAT_RATE = 1.2   # Degrees per time step when heater is ON. Was 0.5.
        self.COOL_RATE = 0.7   # Degrees per time step when A/C is ON
        self.NATURAL_DRIFT_RATE = 0.1  # Heat exchange with ambient per time step

    def get_temperature_reading(self):
        """Simulates the sensor reading (the 'telemetry' data)."""
        return self.current_temp

    def apply_dynamics(self):
        """Calculates the temperature change based on control actions and environment."""
        
        # 1. Natural Heat Exchange (Drift towards ambient temp)
        drift = (self.ambient_temp - self.current_temp) * self.NATURAL_DRIFT_RATE
        self.current_temp += drift

        # 2. Control System Effect
        if self.heater_on:
            self.current_temp += self.HEAT_RATE
            print("    -> HEATER ACTIVELY RAISING TEMP.")
        elif self.ac_on:
            self.current_temp -= self.COOL_RATE
            print("    -> A/C ACTIVELY LOWERING TEMP.")
        
        # Add small random noise for realism
        self.current_temp += random.uniform(-0.1, 0.1)

# --- DIGITAL TWIN MODEL (THE VIRTUAL WORLD) ---

class ThermostatTwin:
    """
    The Digital Twin Model. It contains the state and the control logic.
    """
    def __init__(self, initial_temp, set_point):
        self.current_temp = initial_temp
        self.set_point = set_point
        self.last_update_time = time.time()
        print(f"Digital Twin Initialized. Set Point: {self.set_point:.1f}°C")

    def get_control_command(self, is_heater_on, is_ac_on):
        """
        The CORE LOGIC of the twin (On/Off Control with Hysteresis).
        The twin decides what the physical system SHOULD do.
        """
        temp_delta = self.current_temp - self.set_point
        
        # Hysteresis Band (prevents rapid on/off cycling)
        COOLING_THRESHOLD = 1.0
        HEATING_THRESHOLD = -1.0
        OFF_COOLING_BAND = -0.5
        OFF_HEATING_BAND = 0.5
        
        # Turn ON logic
        if not is_ac_on and temp_delta > COOLING_THRESHOLD:
            return "TURN_AC_ON"
        if not is_heater_on and temp_delta < HEATING_THRESHOLD:
            return "TURN_HEATER_ON"
            
        # Turn OFF logic
        if is_ac_on and temp_delta < OFF_COOLING_BAND:
            return "TURN_OFF"
        if is_heater_on and temp_delta > OFF_HEATING_BAND:
            return "TURN_OFF"
            
        # If no explicit command is given, maintain status quo
        return "NO_CHANGE"
        
    def update_state(self, new_temp):
        """Receives data from the 'Physical Asset' and updates the twin's state."""
        self.current_temp = new_temp
        self.last_update_time = time.time()



In [3]:
# --- MAIN SIMULATION LOOP ---

# 1. Setup the initial conditions
SET_POINT = 15.0
AMBIENT_TEMP = 15.0

# 2. Instantiate the Physical Asset and the Digital Twin
room = PhysicalRoom(initial_temp=20.0, ambient_temp=AMBIENT_TEMP)
twin = ThermostatTwin(initial_temp=room.current_temp, set_point=SET_POINT)

print(f"\n--- Starting Control Simulation (Set Point: {SET_POINT}°C, Ambient: {AMBIENT_TEMP}°C) ---\n")

for i in range(25): # Run for 25 simulation steps
    print(f"\n[TIME STEP {i+1}]")

    current_physical_temp = room.get_temperature_reading()
    twin.update_state(current_physical_temp)
    print(f"  Physical Temp Read: {current_physical_temp:.2f}°C")

    command = twin.get_control_command(is_heater_on=room.heater_on, is_ac_on=room.ac_on)
    print(f"  Twin Command:       {command}")

    if command == "TURN_HEATER_ON":
        room.heater_on = True
        room.ac_on = False
    elif command == "TURN_AC_ON":
        room.ac_on = True
        room.heater_on = False
    elif command == "TURN_OFF":
        room.heater_on = False
        room.ac_on = False
    
    room.apply_dynamics()
    
    time.sleep(0.5)
    
print("\n--- Simulation Complete ---")

Digital Twin Initialized. Set Point: 15.0°C

--- Starting Control Simulation (Set Point: 15.0°C, Ambient: 15.0°C) ---


[TIME STEP 1]
  Physical Temp Read: 20.00°C
  Twin Command:       TURN_AC_ON
    -> A/C ACTIVELY LOWERING TEMP.

[TIME STEP 2]
  Physical Temp Read: 18.87°C
  Twin Command:       NO_CHANGE
    -> A/C ACTIVELY LOWERING TEMP.

[TIME STEP 3]
  Physical Temp Read: 17.69°C
  Twin Command:       NO_CHANGE
    -> A/C ACTIVELY LOWERING TEMP.

[TIME STEP 4]
  Physical Temp Read: 16.79°C
  Twin Command:       NO_CHANGE
    -> A/C ACTIVELY LOWERING TEMP.

[TIME STEP 5]
  Physical Temp Read: 15.98°C
  Twin Command:       NO_CHANGE
    -> A/C ACTIVELY LOWERING TEMP.

[TIME STEP 6]
  Physical Temp Read: 15.27°C
  Twin Command:       NO_CHANGE
    -> A/C ACTIVELY LOWERING TEMP.

[TIME STEP 7]
  Physical Temp Read: 14.63°C
  Twin Command:       NO_CHANGE
    -> A/C ACTIVELY LOWERING TEMP.

[TIME STEP 8]
  Physical Temp Read: 14.05°C
  Twin Command:       TURN_OFF

[TIME STEP 9]
  Phy